# Imports

In [129]:
import os
import pandas as pd
import numpy as np
import string



# Load datasets and preproccesing 


### Load training data

In [130]:
# def load_data():
# # Folder containing the training data
#     folder_path = 'training-data'

#     train_data = {}
#     train_data['sentence'] = []
#     train_data['language'] = []
#     # List to store dataframes
#     i = 0
#     # Loop through each file in the folder
#     for folderName in os.listdir(folder_path):
#         folderPathName = os.path.join(folder_path, folderName)
#         if os.path.isdir(os.path.join(folder_path, folderName)):
#             code = folderName.split('_')[0][:3]
#             train_data['language'].append(code)
#         for filename in  os.listdir(folderPathName):
#             if filename.endswith('sentences.txt'): 
#                 i +=1
#                 file_path = os.path.join(folderPathName, filename)
#                 # Read the data from the text file
#                 with open(file_path, "r", encoding="utf-8") as file:
#                     for line in file.readlines():
#                         if line.strip() != '':
#                             train_data['sentence'].append(line.strip())

#     return train_data
# train_data = load_data()
# print(train_data['sentence'][5])
# array =np.array(train_data)
# print(len(train_data['sentence']))

def load_data():
    folder_path = 'training-data'

    train_data = {'sentence': [], 'language': []}
    # Loop through each file in the folder
    for folderName in os.listdir(folder_path):
        folderPathName = os.path.join(folder_path, folderName)
        if os.path.isdir(folderPathName):
            code = folderName.split('_')[0][:3]
            for filename in os.listdir(folderPathName):
                if filename.endswith('sentences.txt'): 
                    file_path = os.path.join(folderPathName, filename)
                    # Read the data from the text file
                    with open(file_path, "r", encoding="utf-8") as file:
                        for line in file.readlines():
                            if line.strip() != '':
                                train_data['sentence'].append(line.strip())
                                train_data['language'].append(code)

    return train_data

In [131]:
def load_data2():
# Folder containing the training data
    folder_path = 'training-data'

    # List to store dataframes
    dataframes = []
    language_codes = []
    i = 0
    # Loop through each file in the folder
    for folderName in os.listdir(folder_path):
        folderPathName = os.path.join(folder_path, folderName)
        if os.path.isdir(os.path.join(folder_path, folderName)):
            code = folderName.split('_')[0][:3]
            language_codes.append(code)
        for filename in  os.listdir(folderPathName):
            if filename.endswith('sentences.txt'): 
                i +=1
                file_path = os.path.join(folderPathName, filename)
                # Read the data from the text file
                df = open(file_path, 'r', encoding='utf-8').read()
                dataframes.append(df)
    



    return np.array(dataframes), language_codes

## Preprocessing training data
1. **Lower casing and removing accents**
2. **Removing puncatation and special characters** `., etc`
3. **tokenazation using `Tri-grams`**



In [132]:
import re
from unidecode import unidecode
def lower_case(text):
    return text.lower()

def remove_non_words(text):
    return ' '.join(re.findall(r'\b[a-zA-Z]+\b', text))

def clean_whitespace(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def latin_unicode(text):
    return unidecode(text)

def preprocess_sentence(train_data):
    for (i, sentence) in enumerate(train_data['sentence']):
        train_data['sentence'][i] = latin_unicode(sentence)
        train_data['sentence'][i] = lower_case(train_data['sentence'][i])
        train_data['sentence'][i] = remove_non_words(train_data['sentence'][i])
        train_data['sentence'][i] = clean_whitespace(train_data['sentence'][i])
    return train_data


def preprocess_data(dataframes):
    # traindataframe = {}
    # testdataframe = {}
    # validationdataframe = {}

    # traindataframe = []
    # testdataframe = []
    # validationdataframe = []
    for i in range(len(dataframes)):
        dataframes[i] = latin_unicode(dataframes[i])
        dataframes[i] = lower_case(dataframes[i])
        dataframes[i] = remove_non_words(dataframes[i])
        dataframes[i] = clean_whitespace(dataframes[i])
        # print(dataframes[i][5:10])
    #     traindataframe.append(dataframes[i][:6000])
    #     testdataframe.append(dataframes[i][6000:8000])
    #     validationdataframe.append(dataframes[i][8000:10000])
    # # return dataframes, traindataframe, testdataframe, validationdataframe
    # return dataframes, np.array(traindataframe), np.array(testdataframe), np.array(validationdataframe)
    return dataframes


## Encoding of n-grams using d-dimension {+1,-1} distributed representation
- Use of n = 3 `tri grams`

![Alt text]./h-calculation.png)


In [133]:
d = 100
alphabet = ' abcdefghijklmnopqrstuvwxyz'
# item_memory = np.random.choice([-1, 1], size=(len(alphabet), d))
n_gram_size = 3
item_memory = np.random.choice([-1, 1], size=(len(alphabet), d))
permutations = [np.random.choice([-1, 1], size=d) for _ in range(n_gram_size)]


In [134]:
def generate_ngrams(text, n=3):
    ngrams = [text[i:i+n_gram_size] for i in range(len(text) - n_gram_size + 1)]
    return ngrams

def encode_ngram(ngram):
        encoded_vector = np.ones(d)

        for j, char in enumerate(ngram):
            if char == ' ':
                index = 0  
            elif char.isalpha():
                index = ord(char.lower()) - ord('a') + 1  
            else:
                continue  

            # Handle out-of-bounds index (if any)
            if 0 <= index < len(alphabet):
                permuted_vector = item_memory[index] * permutations[j]
                encoded_vector *= permuted_vector
        return encoded_vector    


def calculate_vector(texts, d):
# Initialize item memory and permutations
    ngrams = generate_ngrams(texts)
    bundled_vector = np.zeros(d)
    for ngram in ngrams:
        ngram_vector = encode_ngram(ngram)
        bundled_vector += ngram_vector
    norm = np.linalg.norm(bundled_vector)
    return bundled_vector / norm if norm != 0 else bundled_vector



### Conventional Local Representation Size
The size of a vector in a conventional (local) representation depends on the number of possible n-grams in the given alphabet. The formula for this is \( S = a^n \), where \( a \) is the size of the alphabet and \( n \) is the order of the n-gram.

For example, if we are using the Latin alphabet (26 letters) and \( n = 3 \) (trigrams), the size of the vector is:

\[
S = 27^3 = 19,683
\]

### Difficulties of Working with Conventional Representations of n-Grams in Machine Learning
As mentioned earlier, the size of the one-hot encoded vector for trigrams in the Latin alphabet is \( 19,683 \). This high dimensionality poses an obvious computational bottleneck due to the rapid exponential growth of the number of possible n-grams as \( n \) increases.

Additionally, these vectors are typically sparse, meaning most entries are zero. This sparsity can lead to inefficiencies in both memory usage and processing time. Furthermore, the large vector size increases the risk of overfitting, as models may learn to memorize the training data instead of generalizing to new, unseen data.



## Classification using hyperdimensional centroids

### Load test data

In our test the sentences in the test data is at least 61 symbols long.

In [135]:
import re
import nltk
from nltk.tokenize import sent_tokenize
from unidecode import unidecode
nltk.download('punkt_tab')


def preprocess_test_data(text):
    # Step 1: Remove metadata tags like <SPEAKER ...> and <P>
    text_no_tags = re.sub(r'<[^>]+>', '', text)

    text = latin_unicode(text_no_tags)
    text = lower_case(text)
    text = sent_tokenize(text)

    text_pre = []
    for sentence in text:
        sentence = remove_non_words(sentence)
        sentence = clean_whitespace(sentence)
        text_pre.append(sentence)

    return text_pre


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\boink\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [136]:
import glob
def extract_text_from_file(file_path):
    try:
        with open(file_path, encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

def load_specific_test_data(test_dir, languages):
    # List to store dataframes
    lines_per_language = 100
    languages_codes_2digits = {'bul': 'bg', 'ces': 'cs', 'dan': 'da', 'deu': 'de', 'ell': 'el', 'eng': 'en', 'est': 'et', 'fin': 'fi', 'fra': 'fr', 'hun': 'hu', 'ita': 'it', 'lav': 'lv', 'lit': 'lt', 'nld': 'nl', 'pol': 'pl', 'por': 'pt', 'ron': 'ro', 'slk': 'sk', 'slv': 'sl', 'spa': 'es', 'swe': 'sv'}
    test_data = []
    # Loop through each file in the folder
    for lang_idx, lang in enumerate(languages, start=1):
        language_dir = os.path.join(test_dir, languages_codes_2digits[lang])
        file_pattern = os.path.join(language_dir, "ep-*.txt")
        file_paths = glob.glob(file_pattern)

        total_rows = 0
        language_data = []
        for file_idx, file_path in enumerate(file_paths, start=1):
            text = extract_text_from_file(file_path)
            if text:
                newtext = preprocess_test_data(text)
                newtext = [item for item in newtext if len(item) > 61]
                if total_rows + len(newtext) > lines_per_language:
                    newtext = newtext[:lines_per_language - total_rows]
                    language_data = language_data + newtext
                    break
                language_data = language_data + newtext
                total_rows += len(newtext)
        if len(language_data) != lines_per_language:
            print(f"Warning: Language {lang} has {len(language_data)} lines instead of 1000")
            if len(language_data) == 1:
                print(language_data)
        test_data.append({'text': language_data, 'language': lang})
    
    return pd.DataFrame(test_data)


In [137]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

def similarity_measure(X, V):
    # Calculate the dot product X · V
    dot_product = np.dot(X, V)
    
    norm_X = np.linalg.norm(X)
    norm_V = np.linalg.norm(V)

    similarity = dot_product / (norm_X * norm_V)
    
    return similarity

def classify_language(test_sentence, languages,centroids, d):
    test_vector = calculate_vector(test_sentence, d)

    similarities = [similarity_measure(test_vector, centroid) for centroid in centroids]

    return languages[np.argmax(similarities)]


In [138]:
def prediction_accuracy(predicted_labels_all, test_labels, language_codes):
    test_labels = np.repeat(test_labels, -(-len(predicted_labels_all) // len(test_labels)))
    test_labels = test_labels[:len(predicted_labels_all)]  

    cm = confusion_matrix(test_labels, predicted_labels_all, labels=language_codes)
    accuracy = accuracy_score(test_labels, predicted_labels_all)
    f1 = f1_score(test_labels, predicted_labels_all, average='weighted')

    # Print results
    print(f'Accuracy: {accuracy*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')
    return cm

In [139]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

def confusion_matrix_heatmap(cm, language_codes):
    # Create a confusion matrix heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=language_codes,
                yticklabels=language_codes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

In [ ]:
from typing import OrderedDict


def test(dataframe_pre, language_codes, test_data_pre, d):
    language_centroids = OrderedDict((lang, np.zeros(d)) for lang in language_codes)
    
    for text, lang in zip(dataframe_pre, language_codes):
        # language_centroids[lang] += normalize_vector(calculate_vector(text, d))
        language_centroids[lang] += calculate_vector(text, d)
    
    # print("language_centroids ", language_centroids)
    centroids = np.array(list(language_centroids.values()))
    languages = list(language_centroids.keys())
    predicted_labels = np.array([classify_language(text,languages,centroids, d) for text in test_data_pre])
    return predicted_labels


dataframes, language_codes2 = load_data2()
testload = load_data()
# dataframes = testload['sentence']
# language_codes = testload['language']
dataframe_pre2 = preprocess_data(dataframes)
dataframe_pre = preprocess_sentence(testload)

# print('dataframe ', dataframe_pre['sentence'][200901])

# traindata = []



def split_sentences_by_language(train_data):
    split_data = {}

    # Get a list of unique languages
    unique_languages = set(dataframe_pre['language'])

    for language in unique_languages:
        # Get the sentences for the current language
        language_sentences = [dataframe_pre['sentence'][i] for i in range(len(dataframe_pre['sentence'])) if dataframe_pre['language'][i] == language]

        # Ensure we have 10000 sentences for each language
        if len(language_sentences) == 10000:
            # Split the sentences into 3 groups: 6000, 2000, 2000
            group1 = language_sentences[:6000]
            group2 = language_sentences[6000:8000]
            group3 = language_sentences[8000:]

            group1_text = ' '.join(group1)

            # Store the groups for this language
            split_data[language] = {
                "trainingdata": group1_text,
                "testingdata": group2,
                "validationdata": group3
            }

    return split_data
print("before split")
split_data = split_sentences_by_language(dataframe_pre)
print("after split")
traindatapre = [split_data[lang]['trainingdata'] for lang in split_data]
traindatapre = np.array(traindatapre)
print("dataframe_pre", len(traindatapre[0]))
test_data_pre = [split_data[lang]['testingdata'] for lang in split_data]
print("test_data_pre", len(test_data_pre[0]))
language_codes = list(split_data.keys())
print("language_codes", language_codes)
print("dataframe_pre2", len(dataframe_pre2[0]))
print("dataframe_pre2first", dataframe_pre2[1])
print("dataframe_pre2first", dataframe_pre2[2])

# print(len(split_data['eng']['group1']))  # First group of English sentences
# print(len(split_data['eng']['group2']))  # Second group of English sentences
# print(len(split_data['eng']['group3']))  # Third group of English sentences

# traindataframe = dataframe_pre[:6000]
# testdataframe = dataframe_pre[6000:8000]
# validationdataframe = dataframe_pre[8000:10000]

# for i in range(len(dataframe_pre)):
#     traindataframe[i] = dataframe_pre[i][:6000]
# traindataframetest = ' '.join(traindataframe)

# print("traindataframe", traindataframetest[0])
# print("testdataframe", traindataframe)


# dataframe_pre, traindataframe, testdataframe, validationdataframe = preprocess_data(dataframes)
# print("test data", traindataframe[20], testdataframe[20], validationdataframe[20])
# print("dataframe", dataframe_pre[20])
# print("traindata", traindataframe)

test_data2 = load_specific_test_data('test-data', language_codes)
test_data_pre2 = test_data2['text']
test_data_pre2 = np.array(test_data_pre2)
test_labels = [text[-3:] for text in test_data2['language']]
test_data_pre2 = np.concatenate(test_data_pre2).ravel()
print("test_data_pre2", test_data_pre2['text'], test_data_pre2['language'])

predicted_labels_all = test(traindatapre, language_codes, test_data_pre2, 100)

cm = prediction_accuracy(predicted_labels_all, test_labels, language_codes)
confusion_matrix_heatmap(cm, language_codes)

# # predicted_labels_all2 = test(traindataframe, language_codes, test_data_pre2, 1000)
# cm = prediction_accuracy(predicted_labels_all2, test_labels, language_codes)
# confusion_matrix_heatmap(cm, language_codes)

before split
after split
dataframe_pre 616767
test_data_pre 2000
language_codes ['slk', 'nld', 'pol', 'bul', 'slv', 'fin', 'lav', 'ell', 'ces', 'lit', 'ron', 'dan', 'por', 'fra', 'ita', 'eng', 'deu', 'est', 'hun', 'spa', 'swe']
dataframe_pre2 1072478
dataframe_pre2first diskutujici nesmiji zveoejoovat identifikaeni udaje obiti trestnych einu redakce si vyhrazuje pravo mazat poispivky zejmena ty ktere nesouviseji s tematem diskuze a ktere porusuji pravidla diskuze na nataceni sceny koupani v jezirku pro film oldrich a bozena vzpominal s koutky zavesenymi za usima hodin ivysilani ceska televize smrt lisky bystrousky v nemocne kuzi mr fake pianista babis vs slonkova pro prehravani je potreba mit zapnuty javascript kvetna mala muzika nause pepika sejdeme se na hadaccedalsi posezeni s dechovkou sejdeme se na hadacce se chysta ve vyrove v kd na hadacce kde v sobotu dubna vystoupi josef zima a mala muzika honzy kubika prvni pobocky v asii a usa oblast obchodni cinnosti product service produkt

trigram with words: Accuracy: 4.38 %
F1 Score: 1.57 %
trigram with letters:
Accuracy: 5.14 %
F1 Score: 2.06 %